In [ ]:
%matplotlib inline


# Fairness - Performance: XGB

PiML model diagnostics


Train model and Run 



In [ ]:
from piml import Experiment
from piml.models import PiXGBClassifier
import pandas as pd

exp = Experiment()
data = pd.read_csv('https://github.com/SelfExplainML/PiML-Toolbox/blob/main/datasets/CreditSimuBalanced.csv?raw=true')
exp.data_loader(data)
exp.data_summary(feature_exclude=['Race', 'Gender'])
exp.data_prepare(target='Status', task_type='Classification')
train_x, train_y, _ = exp.get_data(train=True)
xgb2_mono = PiXGBClassifier(max_depth=2, n_estimators=100, feature_names=exp.get_feature_names(),
                            mono_increasing_list=['Mortgage', 'Balance'],
                            mono_decreasing_list=['Amount Past Due', 'Utilization', 'Delinquency Status', 'Credit Inquiry', 'Open Trade'])
xgb2_mono.fit(train_x, train_y)
xgb2_wrapper = exp.make_pipeline(xgb2_mono)
exp.register(xgb2_wrapper, name='XGB2_monotonic')

Fairness Metric



In [ ]:
exp.model_fairness(model='XGB2_monotonic', show='metrics', metric='AIR', group_category=['Race','Gender'],
           reference_group=[1., 1.], protected_group=[0., 0.],
           favorable_threshold=0.5, performance_metric='ACC')

Fairness Segmented



In [ ]:
exp.model_fairness(model='XGB2_monotonic', show='segmented', metric='AIR',
           segmented_feature='Balance', group_category=['Race','Gender'],
           reference_group=[1., 1.], protected_group=[0., 0.],
           favorable_threshold=0.5, segmented_bins=5)

Fairness Thresholding



In [ ]:
exp.model_fairness(model='XGB2_monotonic', show='thresholding', metric='AIR', 
                   group_category=['Race','Gender'], reference_group=[1., 1.],
                   protected_group=[0., 0.], favorable_threshold=0.32, performance_metric='ACC')

Fairness Binning



In [ ]:
binning_list = [{'Balance':{'type': 'customize', 'value': (0, 0.25)}}]
exp.model_fairness(model='XGB2_monotonic', show='binning', metric='AIR', 
                   group_category=['Race','Gender'], reference_group=[1., 1.], protected_group=[0., 0.],
                   favorable_threshold=0.5, performance_metric='F1', binning_list=binning_list)

Fairness Feature Select



In [ ]:
exp.model_fairness(model='XGB2_monotonic', show='removal',metric='AIR',
                   group_category=['Race','Gender'], reference_group=[1., 1.],
                   protected_group=[0., 0.], favorable_threshold=0.5, performance_metric='F1')